# Boston Housing

In [38]:
include("../src/load.jl")
using AlgorithmicRecourse, MLDatasets, Flux
using Plots, PlotThemes
theme(:juno)
using Logging
disable_logging(Logging.Info)
output_folder = "output/boston_housing_ensemble"

"output/boston_housing_ensemble"

## Training the classifier

In [39]:
using MLDatasets, Statistics
X = BostonHousing.features()
y = BostonHousing.targets()
y = Float64.(y .>= median(y)); # binary target

To start off we will just train a single neural network for the binary classification task.

In [40]:
# Prepare data and model:
using Random
Random.seed!(1234)
using StatsBase
dt = fit(ZScoreTransform, X, dims=2)
StatsBase.transform!(dt, X)
xs = Flux.unstack(X,2)
data = zip(xs,y)
nn = Models.build_model(input_dim=size(X)[1], n_hidden=100)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y)

loss (generic function with 1 method)

The model achieves decent training accuracy

In [41]:
run = false
if run
  # Train model:
  using Flux.Optimise: update!, ADAM
  using Statistics, StatsBase
  opt = ADAM()
  epochs = 100
  avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
  accuracy(data) = sum(map(d ->round.(Flux.σ.(nn(d[1]))) .== d[2], data))[1]/length(data)

  using Plots
  anim = Animation()
  avg_l = [avg_loss(data)]
  p1 = scatter( ylim=(0,avg_l[1]), xlim=(0,epochs), legend=false, xlab="Epoch", title="Average loss")
  acc = [accuracy(data)]
  p2 = scatter( ylim=(0.5,1), xlim=(0,epochs), legend=false, xlab="Epoch", title="Accuracy")

  for epoch = 1:epochs
    for d in data
      gs = gradient(params(nn)) do
        l = loss(d...)
      end
      update!(opt, params(nn), gs)
    end
    avg_l = vcat(avg_l,avg_loss(data))
    plot!(p1, [0:epoch], avg_l, color=1)
    scatter!(p1, [0:epoch], avg_l, color=1)
    acc = vcat(acc,accuracy(data))
    plot!(p2, [0:epoch], acc, color=1)
    scatter!(p2, [0:epoch], acc, color=1)
    plt=plot(p1,p2, size=(600,300))
    frame(anim, plt)
  end

  gif(anim, "www/boston_housing_single_nn.gif", fps=10);
end

![](www/boston_housing_single_nn.gif)

Next we will build and train a deep ensemble.

In [42]:
opt = ADAM()
loss_type = :logitbinarycrossentropy
run = false
if run
    K = 50
    𝓜 = Models.build_ensemble(K,kw=(input_dim=size(X)[1], n_hidden=100));
    𝓜, anim = Models.forward(𝓜, data, opt, n_epochs=30, plot_every=10, loss_type=loss_type); # fit the ensemble
    Models.save_ensemble(𝓜, root=output_folder) # save to disk
    gif(anim, "www/boston_housing_ensemble_loss.gif", fps=25);
end

![](www/boston_housing_ensemble_loss.gif)

Testing a single run:

In [43]:
# 𝓜 = Models.load_ensemble(root=output_folder)
# 𝑴 = Models.FittedEnsemble(𝓜,opt,loss_type);
# using Random
# Random.seed!(1234)
# x̅ = X[:,(y.==0)'][:,rand(1:length((y.==0)'))] # select individual sample
# x̅ = reshape(x̅, (length(x̅),1))
# γ = 0.75
# target=1.0
# T = 1000
# n = round(size(X)[2])
# δ = 0.1
# generator = GreedyGenerator(δ,n,:logitbinarycrossentropy,nothing)
# recourse = generate_recourse(generator, x̅, 𝑴, target, γ, T=T); # generate recourse

## Experiment

Prepare ensemble for use with AlgorithmicRecourse.jl:

In [44]:
𝓜 = Models.load_ensemble(root=output_folder)
𝑴 = Models.FittedEnsemble(𝓜, opt, loss_type);
target=1.0
T = 1000
n = round(size(X)[2])
δ = 0.1
generator = GreedyGenerator(δ,n,:logitbinarycrossentropy,nothing)

GreedyGenerator(0.1, 506, :logitbinarycrossentropy, nothing)

Prepare grid of variables for experiment:

In [45]:
# Variables:
μ = [0.01,0.05,0.1]
γ = [0.55,0.75,0.9]
grid_ = Experiments.GridVariables(μ, γ)
n_rounds = 10
# Experiment:
experiment = Experiments.Experiment(X,y,𝑴,target,grid_,n_rounds);

In [47]:
outcome = Experiments.run_experiment(experiment, generator, 5, T=T)
using DataFrames, CSV
CSV.write("output/boston_housing_outcome_greedy.csv", DataFrame(outcome))

18-element Vector{Any}:
 (pct_valid = 1.0, avg_cost = 4.631414470763766, t = 1, μ = 0.01, γ = 0.55, k = 1)
 (pct_valid = 1.0, avg_cost = 5.181698563212648, t = 2, μ = 0.01, γ = 0.55, k = 1)
 (pct_valid = 1.0, avg_cost = 6.40156230931169, t = 1, μ = 0.05, γ = 0.55, k = 1)
 (pct_valid = 0.9615384615384616, avg_cost = 8.03865660418456, t = 2, μ = 0.05, γ = 0.55, k = 1)
 (pct_valid = 1.0, avg_cost = 7.118286310622804, t = 1, μ = 0.1, γ = 0.55, k = 1)
 (pct_valid = 1.0, avg_cost = 9.565563234854498, t = 2, μ = 0.1, γ = 0.55, k = 1)
 (pct_valid = 1.0, avg_cost = 3.0133038346638723, t = 1, μ = 0.01, γ = 0.75, k = 1)
 (pct_valid = 1.0, avg_cost = 5.253570214625479, t = 2, μ = 0.01, γ = 0.75, k = 1)
 (pct_valid = 1.0, avg_cost = 6.355312738174261, t = 1, μ = 0.05, γ = 0.75, k = 1)
 (pct_valid = 1.0, avg_cost = 8.717224328879006, t = 2, μ = 0.05, γ = 0.75, k = 1)
 (pct_valid = 1.0, avg_cost = 10.794906206169653, t = 1, μ = 0.1, γ = 0.75, k = 1)
 (pct_valid = 1.0, avg_cost = 12.819906395914133, t